In [0]:
%pip install sgp4

In [0]:
import requests
from sgp4.api import Satrec, jday
from datetime import datetime
import csv

# --------------------------
# 1. Download TLE file
# --------------------------
url = "https://celestrak.org/NORAD/elements/gps-ops.txt"
try:
    response = requests.get(url, timeout=10)
    response.raise_for_status()
    tle_text = response.text
    with open("gps_ops.tle", "w") as f:
        f.write(tle_text)
    print("TLE data downloaded successfully!")
except requests.exceptions.RequestException as e:
    print("Error downloading TLE data:", e)
    exit(1)

# --------------------------
# 2. Clean TLE lines
# --------------------------
tle_lines = [line.strip() for line in tle_text.splitlines() if line.strip()]
print(f"Total lines after cleaning: {len(tle_lines)}")

# --------------------------
# 3. Compute current Julian date
# --------------------------
utc_now = datetime.utcnow()
jd, fr = jday(utc_now.year, utc_now.month, utc_now.day,
              utc_now.hour, utc_now.minute, utc_now.second + utc_now.microsecond*1e-6)

# --------------------------
# 4. Propagate satellites
# --------------------------
positions = []

for i in range(0, len(tle_lines), 3):
    if i + 2 >= len(tle_lines):
        print(f"Incomplete TLE entry at index {i}, skipping.")
        continue
    
    name = tle_lines[i]
    line1 = tle_lines[i+1]
    line2 = tle_lines[i+2]
    
    sat = Satrec.twoline2rv(line1, line2)
    e, r, v = sat.sgp4(jd, fr)
    
    if e == 0:  # propagation success
        positions.append({
            'name': name,
            'x_km': r[0],
            'y_km': r[1],
            'z_km': r[2],
            'vx_km_s': v[0],
            'vy_km_s': v[1],
            'vz_km_s': v[2]
        })
    else:
        print(f"Propagation error for {name}, error code {e}")

print(f"Successfully propagated {len(positions)} satellites.")

# --------------------------
# 5. Save to CSV
# --------------------------
csv_file = "gps_positions.csv"
with open(csv_file, "w", newline="") as f:
    writer = csv.DictWriter(f, fieldnames=['name','x_km','y_km','z_km','vx_km_s','vy_km_s','vz_km_s'])
    writer.writeheader()
    for sat_info in positions:
        writer.writerow(sat_info)

print(f"Satellite positions saved to {csv_file}")


In [0]:

#Roughly calculating my co-ordnates

import requests
from sgp4.api import Satrec, jday
from datetime import datetime, timezone
import numpy as np

# ----------------------------
# 1. Download TLE data
# ----------------------------
url = "https://celestrak.org/NORAD/elements/gps-ops.txt"
response = requests.get(url, timeout=10)
tle_text = response.text

# Split TLEs into blocks of 3 lines (name, line1, line2)
tle_lines = tle_text.strip().splitlines()
tle_blocks = []
for i in range(0, len(tle_lines), 3):
    name = tle_lines[i]
    line1 = tle_lines[i+1]
    line2 = tle_lines[i+2]
    tle_blocks.append((name, line1, line2))

# ----------------------------
# 2. Define ECI → ECEF
# ----------------------------
def eci_to_ecef(r_eci, dt_utc):
    Y = dt_utc.year
    M = dt_utc.month
    D = dt_utc.day
    H = dt_utc.hour
    Min = dt_utc.minute
    S = dt_utc.second + dt_utc.microsecond*1e-6
    JD = 367*Y - int((7*(Y + int((M+9)/12)))/4) + int((275*M)/9) + D + 1721013.5 + (H + Min/60 + S/3600)/24
    T = (JD - 2451545.0)/36525
    GMST = 280.46061837 + 360.98564736629*(JD-2451545) + 0.000387933*T**2 - T**3/38710000
    GMST = np.deg2rad(GMST % 360)
    R = np.array([
        [np.cos(GMST), np.sin(GMST), 0],
        [-np.sin(GMST), np.cos(GMST), 0],
        [0, 0, 1]
    ])
    return R @ r_eci

# ----------------------------
# 3. ECEF → lat/lon/alt
# ----------------------------
def ecef_to_geodetic(r_ecef):
    a = 6378.137        # km
    e2 = 0.00669437999014
    x, y, z = r_ecef
    lon = np.arctan2(y, x)
    r = np.sqrt(x**2 + y**2)
    lat = np.arctan2(z, r*(1 - e2))
    for _ in range(5):
        N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
        alt = r/np.cos(lat) - N
        lat = np.arctan2(z, r*(1 - e2*N/(N+alt)))
    return np.rad2deg(lat), np.rad2deg(lon), alt

# ----------------------------
# 4. Propagate satellites and convert
# ----------------------------
dt_utc = datetime.now(timezone.utc)
coords = []

for name, line1, line2 in tle_blocks:
    sat = Satrec.twoline2rv(line1, line2)
    jd, fr = jday(dt_utc.year, dt_utc.month, dt_utc.day,
                  dt_utc.hour, dt_utc.minute, dt_utc.second + dt_utc.microsecond*1e-6)
    e, r, v = sat.sgp4(jd, fr)
    if e == 0:
        r_eci = np.array(r)
        r_ecef = eci_to_ecef(r_eci, dt_utc)
        lat, lon, alt = ecef_to_geodetic(r_ecef)
        coords.append((name, lat, lon, alt))
    else:
        print(f"Propagation error for {name}, code {e}")

# ----------------------------
# 5. Print satellite positions
# ----------------------------
for name, lat, lon, alt in coords:
    print(f"{name}: lat={lat:.2f}, lon={lon:.2f}, alt={alt:.2f} km")


In [0]:
import numpy as np
from sgp4.api import Satrec, jday
from datetime import datetime, timezone

# ----------------------------
# 1. Define ECI → ECEF
# ----------------------------
def eci_to_ecef(r_eci, dt_utc):
    Y, M, D = dt_utc.year, dt_utc.month, dt_utc.day
    H, Min, S = dt_utc.hour, dt_utc.minute, dt_utc.second + dt_utc.microsecond*1e-6
    JD = 367*Y - int((7*(Y + int((M+9)/12)))/4) + int((275*M)/9) + D + 1721013.5 + (H + Min/60 + S/3600)/24
    T = (JD - 2451545.0)/36525
    GMST = 280.46061837 + 360.98564736629*(JD-2451545) + 0.000387933*T**2 - T**3/38710000
    GMST = np.deg2rad(GMST % 360)
    R = np.array([
        [np.cos(GMST), np.sin(GMST), 0],
        [-np.sin(GMST), np.cos(GMST), 0],
        [0, 0, 1]
    ])
    return R @ r_eci

# ----------------------------
# 2. ECEF → lat/lon/alt
# ----------------------------
def ecef_to_geodetic(r_ecef):
    a = 6378.137
    e2 = 0.00669437999014
    x, y, z = r_ecef
    lon = np.arctan2(y, x)
    r = np.sqrt(x**2 + y**2)
    lat = np.arctan2(z, r*(1 - e2))
    for _ in range(5):
        N = a / np.sqrt(1 - e2 * np.sin(lat)**2)
        alt = r/np.cos(lat) - N
        lat = np.arctan2(z, r*(1 - e2*N/(N+alt)))
    return np.rad2deg(lat), np.rad2deg(lon), alt

# ----------------------------
# 3. Download TLE and propagate 4+ satellites
# ----------------------------
import requests
url = "https://celestrak.org/NORAD/elements/gps-ops.txt"
tle_text = requests.get(url).text
tle_lines = tle_text.strip().splitlines()
tle_blocks = [(tle_lines[i], tle_lines[i+1], tle_lines[i+2]) for i in range(0, len(tle_lines), 3)]

dt_utc = datetime.now(timezone.utc)
sat_positions = []

# pick first 6 satellites for multilateration
for name, line1, line2 in tle_blocks[:6]:
    sat = Satrec.twoline2rv(line1, line2)
    jd, fr = jday(dt_utc.year, dt_utc.month, dt_utc.day,
                  dt_utc.hour, dt_utc.minute, dt_utc.second + dt_utc.microsecond*1e-6)
    e, r, v = sat.sgp4(jd, fr)
    if e == 0:
        r_eci = np.array(r)
        r_ecef = eci_to_ecef(r_eci, dt_utc)
        sat_positions.append(r_ecef)
    else:
        print(f"Propagation error for {name}, code {e}")

sat_positions = np.array(sat_positions)

# ----------------------------
# 4. Simulate pseudo-ranges (distances) from a hidden "receiver" position
# ----------------------------
# True receiver coordinates (ECEF) – you can replace with approx guess
true_receiver = np.array([1110, -4800, 4000])  # km

pseudo_ranges = np.linalg.norm(sat_positions - true_receiver, axis=1)

# ----------------------------
# 5. Solve multilateration using least squares
# ----------------------------
def estimate_receiver(sat_positions, pseudo_ranges):
    def func(x):
        return np.linalg.norm(sat_positions - x[:3], axis=1) - x[3] - pseudo_ranges
    # initial guess: center of Earth
    x0 = np.array([0,0,0,0])
    
    from scipy.optimize import least_squares
    res = least_squares(func, x0)
    return res.x[:3]

estimated_ecef = estimate_receiver(sat_positions, pseudo_ranges)
lat, lon, alt = ecef_to_geodetic(estimated_ecef)

print(f"Estimated receiver coordinates:\nLatitude: {lat:.5f}°, Longitude: {lon:.5f}°, Altitude: {alt:.2f} km")


In [0]:
def ecef_to_enu(sat_pos, rec_pos, lat, lon):
    """
    Convert satellite ECEF vector into azimuth, elevation relative to receiver.
    """
    dx, dy, dz = sat_pos - rec_pos

    # Rotation matrix ECEF -> ENU
    slat, clat = np.sin(np.deg2rad(lat)), np.cos(np.deg2rad(lat))
    slon, clon = np.sin(np.deg2rad(lon)), np.cos(np.deg2rad(lon))

    R = np.array([[-slon, clon, 0],
                  [-slat*clon, -slat*slon, clat],
                  [clat*clon, clat*slon, slat]])

    enu = R @ np.array([dx, dy, dz])
    e, n, u = enu

    # Compute azimuth and elevation
    az = np.degrees(np.arctan2(e, n)) % 360
    el = np.degrees(np.arctan2(u, np.sqrt(e**2+n**2)))
    return az, el


In [0]:
print("\nSatellite Directions:")
for i, sat in enumerate(sat_positions):
    az, el = ecef_to_enu(sat, estimated_ecef, lat, lon)
    print(f"Sat {i+1}: Azimuth {az:.1f}°, Elevation {el:.1f}°")
